In [ ]:
# Import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from PIL import Image
import requests
from keras.utils import to_categorical
from io import BytesIO

# Set the random seed
np.random.seed(1337)




In [ ]:
# Define constants for the number of training and testing examples for each artist
NUM_TRAIN = 50
NUM_TEST = 20

# Create a dataframe for the cleaned dataset
df = pd.read_csv("https://gist.githubusercontent.com/jd1771/4483a72c10b94fb28d4de22ac12d9b80/raw/1abc382362cfac751b3c784563d1afc327371e9b/clean.csv")

# Get a list of the top 10 artist names 
artist_names = df["Artist"].unique()

encoded_dict = {}

# Encode the artist values to a dictionary
for i in range(len(artist_names)):
  encoded_dict[artist_names[i]] = i



print(encoded_dict)




  



{'Albrecht Durer': 0, 'Alfred Freddy Krupa': 1, 'Claude Monet': 2, 'Giovanni Battista Piranesi': 3, 'Henri Matisse': 4, 'Marc Chagall': 5, 'Pablo Picasso': 6, 'Rembrandt': 7, 'Salvador Dali': 8, 'Vincent van Gogh': 9}


In [ ]:
# Loop through each artist and add the images to memory

artist_paintings = {}

for artist in artist_names:

  artist_paintings[artist] = []
  
  # Get the top paintings and an image URL list for each artist
  top_paintings = df[df['Artist']==artist].head(NUM_TRAIN + NUM_TEST)
  img_list =  top_paintings['Link'].tolist()
  
  for i in range(len(img_list)):

    
    # Get the image link
    painting_link = img_list[i]

    # Get the image content
    response = requests.get(painting_link)
    img = Image.open(BytesIO(response.content))

    # Add the image of the painting to the artist dictionary
    artist_paintings[artist].append(img)
    

In [ ]:
X_train, y_train, X_test, y_test = [] , [], [], []

# Apply image transformations and add the paintings to the train/test splits
for artist, paintings in artist_paintings.items():
    for i in range(len(paintings)):
        
        img = paintings[i]

        # Resize the image
        img_resized = img.resize((100,100))

        # Convert the image to an numpy array
        img_arr = np.array(img_resized)

        # In rare cases, the RGB channel is excluded in the 
        # shape of some images. This will crash the neural network
        # if it is not removed. I have only seen 13 examples out of 
        # 800 where this is the case.
        if len(img_arr.shape) != 3 or img_arr.shape[2] != 3:
          continue
      
        # Normalize the image array
        img_arr = img_arr.astype('float32')
        img_arr /= 255.0

        # Append the painting to the training split
        if i < NUM_TRAIN:
          
          X_train.append(img_arr)
          y_train.append(artist)

        # Append the painting to the test split
        else:
          
          X_test.append(img_arr)
          y_test.append(artist)

    
# Apply 1-k encoding to the label vectors
y_train = [encoded_dict[label] for label in y_train]
y_train = to_categorical(y_train)
y_test = [encoded_dict[label] for label in y_test]
y_test = to_categorical(y_test)

# Shuffle the data
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))






494 494
199 199


In [ ]:
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras import regularizers
from keras.models import Sequential
import keras

# Create the CNN model
model = Sequential()

# Add the layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(100,100,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# Evaluate the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_test), np.array(y_test)), epochs=10)

Epoch 1/10
16/16 [==============================] - 25s 1s/step - loss: 5.8889 - accuracy: 0.1113 - val_loss: 2.2888 - val_accuracy: 0.1307
Epoch 2/10
16/16 [==============================] - 23s 1s/step - loss: 2.2360 - accuracy: 0.1518 - val_loss: 2.2504 - val_accuracy: 0.1457
Epoch 3/10
16/16 [==============================] - 23s 1s/step - loss: 1.9974 - accuracy: 0.2834 - val_loss: 2.1447 - val_accuracy: 0.2412
Epoch 4/10
16/16 [==============================] - 23s 1s/step - loss: 1.4565 - accuracy: 0.5911 - val_loss: 2.2148 - val_accuracy: 0.2563
Epoch 5/10
16/16 [==============================] - 24s 2s/step - loss: 0.9103 - accuracy: 0.7895 - val_loss: 2.7611 - val_accuracy: 0.2513
Epoch 6/10
16/16 [==============================] - 26s 2s/step - loss: 0.5106 - accuracy: 0.8806 - val_loss: 2.7296 - val_accuracy: 0.2060
Epoch 7/10
16/16 [==============================] - 23s 1s/step - loss: 0.3225 - accuracy: 0.9332 - val_loss: 3.1574 - val_accuracy: 0.2563
Epoch 8/10
16/16 [==